In [5]:
import re
import os
import xml.etree.ElementTree as ET
import mysql.connector

In [6]:
pip install mysql-connector

Note: you may need to restart the kernel to use updated packages.


In [35]:
class Lectura_datos:
    def __init__(self, ruta, nombre):
        self.ruta = ruta
        self.nombre = nombre
        self.ubicacion_archivo = ruta + nombre

    def apertura_txt(self):
        with open(self.ubicacion_archivo, 'r') as file:
            global file_source_txt
            file_source_txt = file.readlines()
        print('Tus datos están listos para ser procesados en la variable file_source')
        return file_source_txt

    def apertura_xml(self):
        global file_source_xml
        file_source_xml = ET.parse(self.ubicacion_archivo)
        print('Tu archivo se ha guardado en la variable file_source_xml')
        return file_source_xml

In [36]:
ejemplo1 = Lectura_datos('../data/','Proyecto1_data_txt.txt')

In [37]:
ejemplo1.apertura_txt()

Tus datos están listos para ser procesados en la variable file_source


['index_sql;Q3;Q4;Q5;Q6;Q7;Q8;Q9;Q11;Q12;Q13;Q14;Q15;Q16;Q17;Q20;Q21;Q22;Q23;Q24;Q25;Q26;Q32;Q33;Q34;Q35;Q41\n',
 '1;Indonesia;Master’s degree;Program/Project Manager;20+ years;null, SQL, C, C++, Java;Python;null,   Notepad++  ,  Jupyter Notebook;A cloud computing platform (AWS, Azure, GCP, hosted notebooks, etc);null, None;Never; Matplotlib ;Under 1 year;  Scikit-learn ;Linear or Logistic Regression, Decision Trees or Random Forests;Manufacturing/Fabrication;1000-9,999 employees;1-2;We are exploring ML methods (and may one day put a model into production);null, Build and/or run the data infrastructure that my business uses for storing, analyzing, and operationalizing data;60,000-69,999;$0 ($USD);;;;;Advanced statistical software (SPSS, SAS, etc.)\n',
 '2;Pakistan;Master’s degree;Software Engineer;1-3 years;Python, C++, Java;Python;null,  PyCharm ,  Jupyter Notebook, Other;A laptop;null, Other;Never; Matplotlib ;I do not use machine learning methods;;;Academics/Education;1000-9,999 emp

In [9]:
ejemplo2 = Lectura_datos('../data/','data_xml.xml',)

In [10]:
ejemplo2.apertura_xml()

Tu archivo se ha guardado en la variable file_source_xml


In [74]:
class Limpieza_datos:
    
    def __init__(self, file_source):
        self.file_source = file_source
        
    def limpiar_txt(self):

        """
        Importamos regex para limpiar el string"""

        #import re
        
        """ Convertimos la lista de strings inicial en una lista de listas, separando por ; """

        for i in range(len(self.file_source)):
            self.file_source[i] = self.file_source[i].split(";")

        """Eliminamos los espacios sobrantes al inicio y final de cada elemento"""

        for i in self.file_source:
            for indice, elemento in enumerate(i):
                i[indice] = elemento.strip()

        """ Reemplazamos los caracteres < por under"""

        for i in self.file_source:
            for indice, elemento in enumerate(i):
                i[indice] = elemento.replace("<", "under")

        """ Reemplazamos null por NULL"""

        for i in self.file_source:
            for indice, elemento in enumerate(i):
                i[indice] = elemento.replace("null", "NULL")

        """ Eliminamos los saltos de línea"""

        for i in self.file_source:
            for indice, elemento in enumerate(i):
                i[indice] = elemento.replace("\n", "")

        """ Al separar por ';' hemos obtenido strings vacíos que parecen corresponder a preguntas no respondidas. """

        for i in self.file_source:
            for indice, elemento in enumerate(i):
                if elemento == '':
                    i[indice] = elemento.replace('', 'NULL')

        """ Eliminamos los espacios sobrantes dentro de un mismo elemento"""

        patron_espacios = "\s*,\s+"
        
        for i in self.file_source:
            for indice, elemento in enumerate(i):
                i[indice] = re.sub(patron_espacios, ", ", elemento)

        """ Eliminamos comas en números"""

        numeros = "0123456789"

        for i in self.file_source:
            for indice, elemento in enumerate(i):
                if elemento[0] in numeros or elemento[0] == "$":
                    i[indice] = elemento.replace(",", ".")


        for indice, elemento in enumerate(self.file_source):
            self.file_source[indice] = tuple(elemento)

        global txt_limpio
        txt_limpio = self.file_source
        
        global columnas_txt
        global values_txt
        columnas_txt = txt_limpio[0]
        values_txt = txt_limpio[1:]
        """ El archivo limpio se devuelve en forma de lista de tuplas"""

        print('Tus valores limpios están en values_txt y tus columnas están en columnas_txt')
        return columnas_txt, values_txt
    
    def limpieza_xml(self):
        tree = self.file_source
        root = tree.getroot()
        global xml_list
        xml_list = []
        for child in root:
            xml_dict = {}
            for subchild in child:
                xml_dict[subchild.tag] = subchild.text
            xml_list.append(xml_dict)
              
        for i in xml_list:
            i.pop("level_0")
    
       
        
        for i in xml_list:
            
            if  i["gender"] == "0":
                i["gender"] = "man"
        
            elif i["gender"] == "1":
                i["gender"] = "woman"
        
            elif i["gender"] == "2":
                i["gender"] = "nonbinary"
        
            elif i["gender"] == "3":
                i["gender"] = "prefer not to say"
        
            elif i["gender"] == "4":
                i["gender"] = "prefer to self-describe"

        count = 1
        for i in xml_list:
            i.setdefault("index_sql", count)
            count +=1
        
        
        global xml_limpio
        global columnas_xml
        xml_limpio = []
        columnas_xml = list(xml_list[0].keys())
        if columnas_xml[0] == "index":
            columnas_xml[0] = "index_xml"
        columnas_xml = tuple(columnas_xml)
        

        for i in xml_list:
            i["index"] = int(i["index"])
    
            xml_limpio.append(tuple(i.values()))
            
        print('Tus valores limpios están en xml_limpio y tus columnas están en columnas_xml')
        return xml_limpio, columnas_xml

In [71]:
xml_listo = Limpieza_datos(file_source_xml)

In [72]:

xml_listo.limpieza_xml()


Tus valores limpios están en xml_limpio y tus columnas están en columnas_xml


([(1, '784', '50-54', 'man', 1),
  (2, '924', '22-24', 'man', 2),
  (3, '575', '45-49', 'man', 3),
  (4, '781', '45-49', 'man', 4),
  (5, '1020', '25-29', 'woman', 5),
  (6, '141', '18-21', 'woman', 6),
  (7, '484', '30-34', 'man', 7),
  (8, '1744', '22-24', 'man', 8),
  (9, '655', '30-34', 'man', 9),
  (10, '1777', '40-44', 'man', 10),
  (11, '3081', '18-21', 'woman', 11),
  (12, '1922', '18-21', 'woman', 12),
  (13, '852', '45-49', 'man', 13),
  (14, '838', '22-24', 'man', 14),
  (15, '563', '35-39', 'man', 15),
  (16, '1315', '50-54', 'man', 16),
  (17, '479', '18-21', 'man', 17),
  (18, '249', '22-24', 'man', 18),
  (19, '650', '30-34', 'man', 19),
  (20, '1461', '70+', 'man', 20),
  (21, '551', '25-29', 'woman', 21),
  (22, '258', '30-34', 'man', 22),
  (23, '773', '35-39', 'man', 23),
  (24, '630', '18-21', 'man', 24),
  (25, '2467', '25-29', 'woman', 25),
  (26, '7750', '45-49', 'man', 26),
  (27, '607', '22-24', 'man', 27),
  (28, '525', '22-24', 'woman', 28),
  (29, '501', '18

In [73]:
columnas_xml

('index_xml', 'time', 'age', 'gender', 'index_sql')

# Insercion Datos

In [44]:
def insercion_datos(base_datos, pwd, tabla, columnas, valores):
    conexion = mysql.connector.connect(user='root',
                            password=pwd,
                            host='127.0.0.1',
                            database=base_datos)
    mycursor = conexion.cursor()
    
    query = f'INSERT INTO {tabla} {columnas} VALUES {valores[:]}'
    
    
    print(query) 
    
    try:
        mycursor.execute(query)
        conexion.commit()
        print(mycursor.rowcount, "registro/s insertado/s.")
          
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)
    
    mycursor.close()
    conexion.close()
    
 

In [45]:
insercion_datos("lesicadata", "AlumnaAdalab", "tabla_xml", columnas_xml, xml_limpio)

INSERT INTO tabla_xml ('index_xml', 'time', 'age', 'gender', 'index_sql') VALUES [('1', '784', '50-54', 'man', 1), ('2', '924', '22-24', 'man', 2), ('3', '575', '45-49', 'man', 3), ('4', '781', '45-49', 'man', 4), ('5', '1020', '25-29', 'woman', 5), ('6', '141', '18-21', 'woman', 6), ('7', '484', '30-34', 'man', 7), ('8', '1744', '22-24', 'man', 8), ('9', '655', '30-34', 'man', 9), ('10', '1777', '40-44', 'man', 10), ('11', '3081', '18-21', 'woman', 11), ('12', '1922', '18-21', 'woman', 12), ('13', '852', '45-49', 'man', 13), ('14', '838', '22-24', 'man', 14), ('15', '563', '35-39', 'man', 15), ('16', '1315', '50-54', 'man', 16), ('17', '479', '18-21', 'man', 17), ('18', '249', '22-24', 'man', 18), ('19', '650', '30-34', 'man', 19), ('20', '1461', '70+', 'man', 20), ('21', '551', '25-29', 'woman', 21), ('22', '258', '30-34', 'man', 22), ('23', '773', '35-39', 'man', 23), ('24', '630', '18-21', 'man', 24), ('25', '2467', '25-29', 'woman', 25), ('26', '7750', '45-49', 'man', 26), ('27', 